# Fun with pandas

Examples of using pandas to filter and analyze the data in the wrangled Twitter profiles and tweets data


In [48]:
from os.path import join
import pandas as pd
WRANGLED_DIR = join('data', 'wrangled')
dfprofiles = pd.read_csv(join(WRANGLED_DIR, 'congress-twitter-profiles.csv'))
dftweets = pd.read_csv(join(WRANGLED_DIR, 'congress-tweets.csv'))
# later, we'll join this to the other dataframes
dfmembers = pd.read_csv(join('data', 'legislators-basic-info.csv'))

In [34]:
dfprofiles.head()

,id,screen_name,name,created_at,description,location,url,statuses_count,friends_count,followers_count,verified,profile_image_url,bioguide_id,days_since_creation,days_since_last_tweet,tweets_count,tweets_per_day,recent_tweets_per_day
0,3026622545,RepAmata,Congresswoman Amata,Mon Feb 09 16:00:36 +0000 2015,Congresswoman Representing American Samoa,"Washington, D.C.",http://t.co/L7B6x0yzTS,1,0,382,True,http://pbs.twimg.com/profile_images/5648165541...,R000600,449,449,1,0.00,0.1
1,2964222544,RepHardy,Rep. Cresent Hardy,Tue Jan 06 14:08:31 +0000 2015,Proudly representing Nevada's 4th Congressiona...,"Nevada, USA",http://t.co/RPDLpARNAe,871,403,2255,True,http://pbs.twimg.com/profile_images/5525152827...,H001070,484,0,871,1.80,27.8
2,234128524,RepToddYoung,Rep. Todd Young,Tue Jan 04 22:35:16 +0000 2011,I serve the people of #IN09 in Congress. Conta...,Indiana's 9th District,http://t.co/zPVchT7xlK,2713,476,9578,True,http://pbs.twimg.com/profile_images/7148332431...,Y000064,1946,5,2713,1.39,34.1
3,2856787757,SenJoniErnst,Joni Ernst,Sun Nov 02 13:32:54 +0000 2014,Follow for news and updates from the Office of...,Iowa,http://t.co/zUYK5PQEOd,858,48,10832,True,http://pbs.twimg.com/profile_images/5608591094...,E000295,549,0,858,1.56,70.9
4,2964174789,SenThomTillis,Senator Thom Tillis,Tue Jan 06 14:22:18 +0000 2015,Official Twitter account of North Carolina Sen...,"Washington, D.C.",http://t.co/mLVkKjLyOt,1248,2332,8451,True,http://pbs.twimg.com/profile_images/7174832643...,T000476,484,0,1248,2.58,105.4


In [35]:
dftweets.head()

,id,text,created_at,favorite_count,retweet_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id_str,is_quote_status,bioguide_id,user_id,user_screen_name,source_name,retweeted_status_id,retweeted_status_user_id
0,564820451508383746,"Hello everyone, I'm excited to join Twitter to...",Mon Feb 09 16:17:44 +0000 2015,25,16,NaN,NaN,NaN,False,R000600,3026622545,RepAmata,Twitter Web Client,NaN,NaN
1,727613422553157632,Congrats to students who participated in the 2...,Tue May 03 21:38:52 +0000 2016,1,1,NaN,NaN,NaN,False,H001070,2964222544,RepHardy,Twitter Web Client,NaN,NaN
2,727609591475916802,Congrats to Casey Korder from Parson Elementar...,Tue May 03 21:23:39 +0000 2016,1,1,NaN,NaN,NaN,False,H001070,2964222544,RepHardy,Twitter Web Client,NaN,NaN
3,727590935303958529,Happy #NationalTeacherDay to all of Nevada’s i...,Tue May 03 20:09:31 +0000 2016,0,0,NaN,NaN,NaN,False,H001070,2964222544,RepHardy,Twitter Web Client,NaN,NaN
4,727280635082096640,"For National Small Biz Week, I’m hosting a wor...",Mon May 02 23:36:30 +0000 2016,0,2,NaN,NaN,NaN,False,H001070,2964222544,RepHardy,Twitter Web Client,NaN,NaN


# Just counting

## Dataframe quick counts

In [36]:
# How many tweets total?
len(dftweets)

102434

In [37]:
# How many Twitter users have at least 50000 followers?
len(dfprofiles[dfprofiles.followers_count >= 50000])

31

In [53]:
# How many legislators, by party?
dfmembers['party'].value_counts()

R    301
D    236
I      3
Name: party, dtype: int64

# Aggregations

(This is a nice tutorial on aggregations: [http://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/](http://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/))

Pandas has a whole host of functions for reducing a column of values to a scalar (single value) -- `sum()`, for example, will return the sum of all the numbers for each column in a dataframe. To get the total number of Twitter accounts that follow Congressional accounts:

In [ ]:
dfprofiles['followers_count'].sum()

# Filtering

Examples of how to subset a dataframe. 

For the most part, remember that dataframe operations are _non-mutating_ -- they create a copy of the dataframe, but don't alter the data in the dataframe that you applied a function upon.


## Getting a subset of rows using a conditional query

How to find which Twitter users have tweeted more than 2000 times in their lifetimes, but have not tweeted in the last 60 days?

There are several ways to think of this, such as doing two filters:

1. The twitter users who have more than 2000 tweets
2. Of the users in Step 1, filter for users with days_since_last_tweet > 60

In [38]:
# assign this subset of twitter users to the xdf variable
xdf = dfprofiles[dfprofiles.tweets_count > 2000]

# new subset
ydf = xdf[xdf.days_since_last_tweet > 60]

len(ydf)

2

Or, in a single query, with two conditions -- note the weird, non-Python-convention of using `&` instead of `and` to join the conditions. That only works in pandas-land....so yes, you kind-of have to be mindful of a whole different set of rules when working with pandas. So keep sharp with your debugging.

In [39]:
ydf = dfprofiles[(dfprofiles.tweets_count > 2000) & (dfprofiles.days_since_last_tweet > 60)]

len(ydf)

2

## Getting a subset of columns

Who are those `2` congressmembers in `ydf`? Let's just print it out:

In [40]:
ydf

,id,screen_name,name,created_at,description,location,url,statuses_count,friends_count,followers_count,verified,profile_image_url,bioguide_id,days_since_creation,days_since_last_tweet,tweets_count,tweets_per_day,recent_tweets_per_day
244,14845376,GrahamBlog,Lindsey Graham,Tue May 20 13:57:21 +0000 2008,Twitter feed for U.S. Senator Lindsey Graham o...,"Washington, DC",http://t.co/x7LZALizg6,4243,2151,102048,True,http://pbs.twimg.com/profile_images/2167513251...,G000359,2906,75,4243,1.46,26.9
281,435500714,SenCoonsOffice,Office of Sen. Coons,Tue Dec 13 04:02:30 +0000 2011,"News releases, blog posts, new videos, transcr...",Delaware & Washington,http://t.co/YhxmCF9DHF,2201,107,4230,True,http://pbs.twimg.com/profile_images/1690317750...,C001088,1604,91,2201,1.37,9.4


Too many columns. Let's just pick the most relevant ones. The syntax for picking a subset of columns from a dataframe is to provide a list of column names:

In [44]:
ydf[['screen_name', 'days_since_last_tweet', 'tweets_count', 'recent_tweets_per_day']]

,screen_name,days_since_last_tweet,tweets_count,recent_tweets_per_day
244,GrahamBlog,75,4243,26.9
281,SenCoonsOffice,91,2201,9.4


# Ranking and sorting tweets

## Most retweeted tweets

Of the recent tweets collected per Congressmember, which were the 5 most retweeted tweets?

In [45]:
top5 = dftweets.sort_values('retweet_count', ascending=False).head(5)
top5[['id', 'user_screen_name', 'retweet_count', 'text']]


,id,user_screen_name,retweet_count,text
52064,644266781834477569,RepAnnaEshoo,420587,"RT @POTUS: Cool clock, Ahmed. Want to bring it..."
82355,644212922936393728,RepHuffman,420587,"RT @POTUS: Cool clock, Ahmed. Want to bring it..."
71215,690957272051220483,RepTomRice,187117,RT @NationalZoo: Tian Tian woke up this mornin...
41035,690965124144717824,RepHuizenga,187117,RT @NationalZoo: Tian Tian woke up this mornin...
80145,621312647942832128,RepDannyDavis,102786,RT @POTUS: We could eliminate tuition at every...


As it turns out, those retweets appear to be retweets of more popular retweets:

The tweet at:

[https://twitter.com/RepAnnaEshoo/status/644266781834477569](https://twitter.com/RepAnnaEshoo/status/644266781834477569)

Resolves to this [POTUS tweet](https://twitter.com/POTUS/status/644193755814342656):

<a href="https://twitter.com/POTUS/status/644193755814342656"><img src="assets/images/potus-tweet-644193755814342656.jpg" alt="potus-tweet-644193755814342656.jpg"></a>

Sidenote: you can confirm the redirect using Python and requests:

```py
import requests
xurl = 'https://twitter.com/RepAnnaEshoo/status/644266781834477569'
resp = requests.get(xurl)
resp.url
# 'https://twitter.com/POTUS/status/644193755814342656'
```


## Most retweeted original tweets 

So let's restrict our query to tweets that were _not_ retweets. Another way to specify this query is to filter for tweets in which `retweeted_status_id` is _empty_:

Using [pandas `isnull()` function to filter the array](http://pandas.pydata.org/pandas-docs/stable/missing_data.html):


In [54]:
dfnotretweets = dftweets[dftweets.retweeted_status_id.isnull()]
top5 = dfnotretweets.sort_values('retweet_count', ascending=False).head(5)
top5[['id', 'user_screen_name', 'retweet_count', 'text']]

,id,user_screen_name,retweet_count,text
78936,698880570843992064,SenWarren,17232,I can’t find a clause in the Constitution that...
14167,719155984808177664,SenSanders,10010,Dr. King told us: You judge people on their ch...
14161,719512609016430596,SenSanders,9528,"If college could be tuition free 50 years ago,..."
35362,717004180829954048,repjohnlewis,9246,"48 yrs ago today, my friend, my brother, Dr. M..."
78934,698880741367635970,SenWarren,7860,Abandoning their Senate duties would also prov...


Those seem like more well-known tweeters.

# Joining data

Putting together data that is *physically* separated is one of the most difficult but high return-on-investment data-wrangling skills to learn.

Our data comes from two separate sources: Twitter, and people who care about tracking Congress. The latter group have done the work of identifying which congressmembers have entries in Twitter's dataset, i.e. which Congressmembers have Twitter accounts, and the identity of each account.

Yet there's no direct way to analyze the two data sources together. We can find the top 5 most retweeted tweets, but we can't find that ranking by __party__ or by __chamber__, for instance, because those two attributes have meaning only in Congress dataland.

So it's up to us to explicitly __join__ the data.

If you've used SQL, [joins should be familiar to you](http://2015.padjo.org/tutorials/babynames-and-college-salaries/030-all-about-inner-joins/).

Pandas DataFrame also has a concept of a "join" -- here's the [documentation on the various ways to merge dataframes](http://pandas.pydata.org/pandas-docs/stable/merging.html).


## Joining legislator data with Twitter profile data

Let's start with something simple by asking a simple question: which party has more Twitter followers?

To answer that question, we need to join the __party__ column in `dfmembers` with each corresponding row in `dfprofiles`.

At a minimum, we need two columns: `party` and `followers_count`. Then we can do a group sum to get follower count by party.

### Joining by key

However, we actually need a third column -- this would be a column that both `dfmembers` and `dfprofiles` have in common, such that if a row in `dfmembers` has the same value for this common column as `dfprofiles`, then we know that those two rows should be matched together.

If you've been following along, you probably saw in [WRANGLE-PROFILES.md][WRANGLE-PROFILES.md] that I went out of my way to add a `bioguide_id` field to the wrangled profiles and tweets data, so that each tweet and profile could be easily connected to the legislators data, in which the `bioguide_id` uniquely identifies a congressmember.

First, let's create two mini dataframes: a frame of just `bioguide_id` and `followers_count`, and a frame of just `bioguide_id` and `party`:

In [58]:
dfa = dfprofiles[['bioguide_id', 'followers_count']]
dfb = dfmembers[['bioguide_id', 'party']]

In [60]:
dfa.head()

,bioguide_id,followers_count
0,R000600,382
1,H001070,2255
2,Y000064,9578
3,E000295,10832
4,T000476,8451


In [61]:
dfb.head()

,bioguide_id,party
0,A000055,R
1,A000360,R
2,A000367,R
3,A000368,R
4,A000369,R


We can't just throw them together side by side, so we use the pandas.DataFrame `join()` method to tell it exactly how the two tables should be joined:

In [69]:
dfx = pd.merge(dfa, dfb, on='bioguide_id')

In [72]:
dfx.head()

,bioguide_id,followers_count,party
0,R000600,382,R
1,H001070,2255,R
2,Y000064,9578,R
3,E000295,10832,R
4,T000476,8451,R


You'll notice that the combined dataframe, `dfx`, has fewer rows than `dfmembers`:

In [73]:
len(dfx)

518

This should make sense, because not all congressmembers have Twitter accounts (and not all the people the `dfmembers` file are voting members:

In [82]:
twits = dfmembers[dfmembers.twitter_id.notnull()]
len(twits)

521

Let's get the sum of follower counts by party:

In [78]:
dfx.groupby('party').sum()

,followers_count
party,
D,5135009
I,1939416
R,7955594


Of course, there are more Republican legislators than Democrats, so that's not a huge surprise.

### Top tweets by party

In [89]:
# do the merge
dfm = dfmembers[['party', 'bioguide_id', 'title']]
dft = dftweets[dftweets.retweeted_status_id.isnull()] # filter out retweets
dft = dft[['text', 'id', 'retweet_count', 'bioguide_id', 'user_screen_name']]
dfx = pd.merge(dfm, dft, on='bioguide_id').sort_values('retweet_count', ascending=False)

In [90]:
# republican senators
top5 = dfx[(dfx.party == 'R') & (dfx.title == 'Sen')].head(5)
top5[['id', 'user_screen_name', 'retweet_count', 'text']]

,id,user_screen_name,retweet_count,text
15735,706558214859591681,SenTedCruz,2006,Nancy Reagan will be remembered for her deep p...
15791,690579396336422912,SenTedCruz,1571,"Each life is a gift from God. Without #life, t..."
15727,712645781875003392,SenTedCruz,1198,".@POTUS, if you’re litigating against nuns, yo..."
15745,702259162554703878,SenTedCruz,1123,The Senate Judiciary Committee will not hold h...
44083,710103186716168192,SenMikeLee,1017,My colleagues and I on Judiciary Committee hav...


In [91]:
# republican house
top5 = dfx[(dfx.party == 'R') & (dfx.title == 'Rep')].head(5)
top5[['id', 'user_screen_name', 'retweet_count', 'text']]

,id,user_screen_name,retweet_count,text
6073,551058743073734656,RepJBridenstine,3799,I will not vote for John Boehner. My statemen...
28353,667368356853452800,TGowdySC,3192,Pres Obama says we're afraid of widows &amp; o...
6069,551827969933840384,RepJBridenstine,2016,I congratulate @replouiegohmert &amp; @RepTedY...
6045,557308329878618113,RepJBridenstine,1607,I believe that warriors like Chris Kyle are so...
28281,717793468312121345,TGowdySC,1381,Democrats use Benghazi like it's a four-letter...


In [93]:
# democrat senate
top5 = dfx[(dfx.party == 'D') & (dfx.title == 'Sen')].head(5)
top5[['id', 'user_screen_name', 'retweet_count', 'text']]

,id,user_screen_name,retweet_count,text
78494,698880570843992064,SenWarren,17232,I can’t find a clause in the Constitution that...
78492,698880741367635970,SenWarren,7860,Abandoning their Senate duties would also prov...
78493,698880648614846464,SenWarren,4942,The Senate GOP took an oath just like Dems did...
78496,698880441638518785,SenWarren,4615,.@SenateMajLdr is right that Americans should ...
78510,695326631100080128,SenWarren,4555,Every single House Republican who came to work...


In [94]:
# democrat house
top5 = dfx[(dfx.party == 'D') & (dfx.title == 'Rep')].head(5)
top5[['id', 'user_screen_name', 'retweet_count', 'text']]

,id,user_screen_name,retweet_count,text
40852,717004180829954048,repjohnlewis,9246,"48 yrs ago today, my friend, my brother, Dr. M..."
40936,646362466583191552,repjohnlewis,4397,"I was beaten, my skull was fractured, &amp; I ..."
72496,667097811004039169,RepMarkTakano,4043,"Can’t believe this needs clarifying, but the i..."
40871,706927590687121408,repjohnlewis,3478,"When people tell me nothing has changed, I say..."
40900,689092607159459842,repjohnlewis,2743,"Dr. Martin Luther King, Jr. was my friend, my ..."
